## DS5559 - Project
## Notebook 3 - Make a smaller TFIDF
#### Name: Mengyao Zhang (mz6jv), Runhao Zhao (rz6dg)

## Synopsis
Use case: this notebook creates tfidf_small for further analysis such as PCA and HCA.

## Libraries

In [1]:
import pandas as pd
import sqlite3
import os
import textman as tx
from sklearn.feature_extraction.text import TfidfVectorizer

## Configs

In [2]:
corpus_db = 'project.db'
max_words = 1000  # max number of words queried

num_topics = 20
num_iters = 1000
show_interval = 100

In [3]:
OHCO = ['book_num','chap_num', 'para_num', 'sent_num', 'token_num']
BOOKS = OHCO[:1] 
CHAPS = OHCO[:2]
PARAS = OHCO[:3]
SENTS = OHCO[:4]
#BAG = PARAS 
BAG = CHAPS

## Get tokens we want 
filter out stop words and proper nouns, and only get the top words based on count n

In [4]:
# use SQL to get the tokens we want
sql = """
SELECT * FROM token 
WHERE term_id IN (
    SELECT term_id FROM vocab 
    WHERE stop = 0
    ORDER BY n DESC LIMIT {}
) 
AND (pos NOT LIKE 'NNP%')
""".format(max_words)

In [5]:
with sqlite3.connect(corpus_db) as db:
    tokens = pd.read_sql(sql, db,index_col=BAG)

In [6]:
tokens.head()

para_num  sent_num  token_num  pos token_str  punc  num  \
book_num chap_num                                                            
1        0                1         0          0   NN   Chapter     0    0   
         0                2         0          5  VBG    taking     0    0   
         0                2         0          7   NN      walk     0    0   
         0                2         0          9   NN       day     0    0   
         0                2         1          5   RB    indeed     0    0   

                  term_str  term_id  
book_num chap_num                    
1        0         chapter     6192  
         0          taking    38869  
         0            walk    43412  
         0             day     9790  
         0          indeed    20395

In [7]:
len(tokens.reset_index().groupby(BAG)) 
# fewer chaps than the expected 1622 since many terms are filtered out

1593

In [8]:
# check the number of unique terms
len(tokens.term_id.unique())

985

In [9]:
# gather tokens for each chapter
# the following is for BAG=CHAPS
corpus = tx.gather_tokens(tokens, level=2, col='term_str')\
    .reset_index()#.rename(columns={'term_str':'doc_content'})
#corpus['doc_label'] = corpus.apply(lambda x: "book-{}_chap-{}".format(x.book_num,x.chap_num), 1)

In [10]:
corpus.head()

,book_num,chap_num,term_str
0,1,0,chapter taking walk day indeed hour morning si...
1,1,1,chapter way new thing bad opinion fact rather ...
2,1,2,chapter next thing remember feeling seeing red...
3,1,3,chapter enough hope get well change seemed nea...
4,1,4,chapter five clock hardly struck morning broug...


In [11]:
corpus.shape

(1593, 3)

## Calculate tfidf on corpus

In [12]:
# initiate vectorizer
vectorizer = TfidfVectorizer()
# fit and transform corpus
result = vectorizer.fit_transform(corpus.term_str)

In [13]:
result

<1593x984 sparse matrix of type '<class 'numpy.float64'>'
	with 608440 stored elements in Compressed Sparse Row format>

In [14]:
# change resulted sparse matrix to a df
result = pd.DataFrame(result.toarray())

In [15]:
result.shape 

(1593, 984)

In [16]:
result.head()

,0,1,2,3,4,5,6,7,8,9,...,974,975,976,977,978,979,980,981,982,983
0,0.000000,0.0,0.0,0.037412,0.0,0.000000,0.046796,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.098101,0.000000,0.000000,0.047139,0.025306,0.0
1,0.000000,0.0,0.0,0.027259,0.0,0.034097,0.000000,0.000000,0.0,0.027512,...,0.000000,0.0,0.0,0.000000,0.047653,0.020395,0.000000,0.051520,0.036877,0.0
2,0.019922,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.043675,...,0.000000,0.0,0.0,0.022621,0.018912,0.064753,0.030207,0.054525,0.043907,0.0
3,0.000000,0.0,0.0,0.014575,0.0,0.036462,0.000000,0.020038,0.0,0.000000,...,0.019656,0.0,0.0,0.000000,0.025479,0.010905,0.000000,0.009182,0.039435,0.0
4,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.039071,0.0,0.028683,...,0.000000,0.0,0.0,0.000000,0.024841,0.042525,0.000000,0.035808,0.019223,0.0


In [17]:
corpus[BAG].shape

(1593, 2)

In [18]:
# add index (OHCO) back to result
index_col = corpus[BAG]
tfidf_small =  pd.concat([index_col, result], axis=1)

In [19]:
tfidf_small=tfidf_small.set_index(BAG)

In [20]:
tfidf_small.head()

0    1    2         3    4         5         6    \
book_num chap_num                                                          
1        0         0.000000  0.0  0.0  0.037412  0.0  0.000000  0.046796   
         1         0.000000  0.0  0.0  0.027259  0.0  0.034097  0.000000   
         2         0.019922  0.0  0.0  0.000000  0.0  0.000000  0.000000   
         3         0.000000  0.0  0.0  0.014575  0.0  0.036462  0.000000   
         4         0.000000  0.0  0.0  0.000000  0.0  0.000000  0.000000   

                        7    8         9   ...        974  975  976       977  \
book_num chap_num                          ...                                  
1        0         0.000000  0.0  0.000000 ...   0.000000  0.0  0.0  0.000000   
         1         0.000000  0.0  0.027512 ...   0.000000  0.0  0.0  0.000000   
         2         0.000000  0.0  0.043675 ...   0.000000  0.0  0.0  0.022621   
         3         0.020038  0.0  0.000000 ...   0.019656  0.0  0.0  0.000000   
         4         0.039071  0.0  0.028683 ...   0.000000  0.0  0.0  0.000000   

                        978       979       980       981       982  983  
book_num chap_num                                                         
1        0         0.098101  0.000000  0.000000  0.047139  0.025306  0.0  
         1         0.047653  0.020395  0.000000  0.051520  0.036877  0.0  
         2         0.018912  0.064753  0.030207  0.054525  0.043907  0.0  
         3         0.025479  0.010905  0.000000  0.009182  0.039435  0.0  
         4         0.024841  0.042525  0.000000  0.035808  0.019223  0.0  

[5 rows x 984 columns]

## Add corresponding term_id for features in tfidf_small

In [21]:
features = vectorizer.get_feature_names()

In [23]:
# change to df 
features_df = pd.DataFrame({'term_str':features})

In [24]:
tokens_sub = tokens.reset_index()[['term_str','term_id']]

In [25]:
# drop duplicated terms in df
tokens_sub.drop_duplicates("term_str",inplace=True)

In [27]:
tokens_sub.shape

(985, 2)

In [28]:
# join the two df on 'term_str' so that we can get the corresponding term_id for each term
new_df = features_df.merge(tokens_sub,how="left",on='term_str')

In [29]:
new_df.head()

,term_str,term_id
0,able,95
1,absence,150
2,absolutely,162
3,account,296
4,acquaintance,382


In [30]:
# rename the columns in tfidf_small using terms_id
tfidf_small.columns = new_df['term_id']

In [31]:
tfidf_small.head()

term_id               95     150    162       296    382       415    \
book_num chap_num                                                      
1        0         0.000000    0.0    0.0  0.037412    0.0  0.000000   
         1         0.000000    0.0    0.0  0.027259    0.0  0.034097   
         2         0.019922    0.0    0.0  0.000000    0.0  0.000000   
         3         0.000000    0.0    0.0  0.014575    0.0  0.036462   
         4         0.000000    0.0    0.0  0.000000    0.0  0.000000   

term_id               417       420    447       461    ...       44700  \
book_num chap_num                                       ...               
1        0         0.046796  0.000000    0.0  0.000000  ...    0.000000   
         1         0.000000  0.000000    0.0  0.027512  ...    0.000000   
         2         0.000000  0.000000    0.0  0.043675  ...    0.000000   
         3         0.000000  0.020038    0.0  0.000000  ...    0.019656   
         4         0.000000  0.039071    0.0  0.028683  ...    0.000000   

term_id            44804  44819     44825     44833     44857     44860  \
book_num chap_num                                                         
1        0           0.0    0.0  0.000000  0.098101  0.000000  0.000000   
         1           0.0    0.0  0.000000  0.047653  0.020395  0.000000   
         2           0.0    0.0  0.022621  0.018912  0.064753  0.030207   
         3           0.0    0.0  0.000000  0.025479  0.010905  0.000000   
         4           0.0    0.0  0.000000  0.024841  0.042525  0.000000   

term_id               44864     44911  44929  
book_num chap_num                             
1        0         0.047139  0.025306    0.0  
         1         0.051520  0.036877    0.0  
         2         0.054525  0.043907    0.0  
         3         0.009182  0.039435    0.0  
         4         0.035808  0.019223    0.0  

[5 rows x 984 columns]

In [34]:
tfidf_small.shape

(1593, 984)

## Save data

In [35]:
 with sqlite3.connect(corpus_db) as db:    
    tfidf_small.to_sql('tfidf_small', db, if_exists='replace', index=True)

In [ ]:
# END